In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
%matplotlib inline
import matplotlib.pyplot as plt

box_train = pd.read_csv("../input/train.csv", index_col='id')

**Getting insight of data**

Try to get following information:
* Number of columns and data availability per column
* Type of data in each column
* Understanding meaning of data in each column

In [ ]:
print(box_train.info())

In [ ]:
print(box_train.describe())

In [ ]:
example = 2
for i in range(len(box_train.iloc[example])):
    print(box_train.columns[i],' ==> ', box_train.iloc[example,i])

**There are 6 different types of data available**
1. Numerical
1. String
1. Date
1. Image
1. Categorical
1. URL

**Analysing individual features**

In [ ]:
box_train.loc[:,['original_title','title']].head(10)

In [ ]:
box_train.loc[:,['homepage']].head()

In [ ]:
print(box_train.status.unique())
rumored = box_train[box_train['status'] == 'Rumored']
rumored

In [ ]:
box_train.nunique()

In [ ]:
box_test = pd.read_csv("../input/test.csv", index_col='id')

box = pd.concat([box_train.drop('revenue', axis=1), box_test])
box.info()

**Insight from individual provided features**
1. Original_title can be removed since title contains same information in English.
1. Homepage url does not provide any useful information hence it can be removed too.
1. imdb_id contains unique id to map movie to imdb database.
1. Poster_path column contains image name which can be retrieved from themoviedb.org website. Base url to fetch the image is http://image.tmdb.org/t/p/w185/ . example: http://image.tmdb.org/t/p/w185//nBNZadXqJSdt05SHLqgT0HuC5Gm.jpg.
1. overview, tagline and title contains text information.
1. belongs_to_collection, genres, production_companies, production_countries, Keywords, cast, crew and spoken_language contains information as list of dictionary items.

**Not useful(at least for this implementation) columns will be removed.**

In [ ]:
not_useful_columns = ['homepage', 'imdb_id', 'original_title', 'overview', 'poster_path',
                     'tagline', 'title']
box_train.drop(not_useful_columns, axis=1, inplace=True)
box_train.info()

**Segregating numerical columns**

In [ ]:
num_box = box_train[['budget', 'popularity', 'runtime', 'revenue']]

**Correlation among numerical features with respect to target(revenue)**

    As seen, Revenue looks fairly correlated with budget followed by popularity. Popularity is mildly correlated with revenue.

In [ ]:
import seaborn

corr_mx = num_box.corr()
print(corr_mx)
seaborn.heatmap(corr_mx)
plt.show()

    Let's check value distribution of numerical features.
* Budget and revenue follows almost similar distribution.

In [ ]:
num_box.hist(bins=100, figsize=(16,12))
plt.show()

**Feature engineering**
    
    As we have seen from analysis, provided data contains information in various forms and various datatype. To use those available information, they need to be transformed before we feed it into machine learning model.
1. Data in dectionary format
    * First of all seperate dataframes will be created for each columns which has information in dectionary type. Let's understand it with example of genres column: genres dataframe will have 'id' and 'name' of genres appeared in dataset.
    * Then average popularity and budget will be calculated for each entry in genres dataframe. Average will be calculated over popularity and budget of all movies for which that gerne is appeared.
    * Now, popularity and budget of any movie according to genres will be created by calculating average of popularity and budget of all gernes for that movie.
    * Only top 5 cast will be considered from 'cast' column.
    * Only Directors will be considered from 'crew' column.    
1. 'release_date' will be converted into 3 columns, day, month and year.
1. Categorical values will be converted into One-Hot matrix

In [ ]:
import ast

# Function to transfer unique dictionary value in DataFrame

def get_as_dataframe(df, column):
    as_dataframe = pd.DataFrame()
    for i in range(1,len(df)+1):
        if str(df.loc[i, column]) != 'nan':
            temp_list = ast.literal_eval(df.loc[i, column])
            temp = pd.DataFrame(temp_list)
            as_dataframe = pd.concat([as_dataframe, temp], ignore_index=True)
    return as_dataframe.drop_duplicates()

In [ ]:
def get_as_dataframe_for_top_5_cast(df, column):
    as_dataframe = pd.DataFrame()
    for i in range(1,len(df)+1):
        if str(df.loc[i, column]) != 'nan':
            temp_list = ast.literal_eval(df.loc[i, column])
            if temp_list:
                temp = pd.DataFrame(temp_list)
                temp = temp.iloc[:5,:]
                as_dataframe = pd.concat([as_dataframe, temp], ignore_index=True)
    return as_dataframe.drop_duplicates()

In [ ]:
def get_as_dataframe_for_director(df, column):
    as_dataframe = pd.DataFrame()
    for i in range(1,len(df)+1):
        if str(df.loc[i, column]) != 'nan':
            temp_list = ast.literal_eval(df.loc[i, column])
            if temp_list:
                temp = pd.DataFrame(temp_list)
                temp = temp[temp['job'] == 'Director']
                as_dataframe = pd.concat([as_dataframe, temp], ignore_index=True)
    return as_dataframe.drop_duplicates()

In [ ]:
dataframes = {}

In [ ]:
def prepare_as_dataframe(df, column, index_col):
    temp = get_as_dataframe(df, column)
    print(column,' - length : ',str(len(temp)))
    temp = temp.set_index(index_col).sort_index()
    temp['count'], temp['popularity'], temp['budget'] = 0, 0.0, 0.0
    return temp

In [ ]:
def prepare_belongs_to_collection_as_dataframe(df):
    temp = get_as_dataframe(df, 'belongs_to_collection')
    temp = temp[['id', 'name']]
    print('belongs_to_collection - length : ', str(len(temp)))
    temp = temp.set_index('id').sort_index()
    temp['count'], temp['popularity'], temp['budget'] = 0, 0.0, 0.0
    return temp

In [ ]:
def prepare_top_5_cast_as_dataframe(df):
    temp = get_as_dataframe_for_top_5_cast(df, 'cast')
    temp = temp[['id', 'name']].drop_duplicates()
    temp = temp.set_index('id').sort_index()
    print('cast - length : ', str(len(temp)))
    temp['count'], temp['popularity'], temp['budget'] = 0, 0.0, 0.0
    return temp

In [ ]:
def prepare_director_as_dataframe(df):
    temp = get_as_dataframe_for_director(df, 'crew')
    temp = temp[['id', 'name']].drop_duplicates()
    temp = temp.set_index('id').sort_index()
    print('director - length : ', str(len(temp)))
    temp['count'], temp['popularity'], temp['budget'] = 0, 0.0, 0.0
    return temp

In [ ]:
dataframes['belongs_to_collection'] = prepare_belongs_to_collection_as_dataframe(box)
dataframes['genres'] = prepare_as_dataframe(box, 'genres', 'id')
dataframes['production_companies'] = prepare_as_dataframe(box, 'production_companies', 'id')
dataframes['production_countries'] = prepare_as_dataframe(box, 'production_countries', 'iso_3166_1')
dataframes['spoken_languages'] = prepare_as_dataframe(box, 'spoken_languages', 'iso_639_1')
dataframes['Keywords'] = prepare_as_dataframe(box, 'Keywords', 'id')
dataframes['cast'] = prepare_top_5_cast_as_dataframe(box)
dataframes['crew'] = prepare_director_as_dataframe(box)

In [ ]:
dataframes['belongs_to_collection'].head()

In [ ]:
dataframes['genres'].head()

In [ ]:
dataframes['production_companies'].head()

In [ ]:
dataframes['production_countries'].head()

In [ ]:
dataframes['spoken_languages'].head()

In [ ]:
dataframes['Keywords'].head()

In [ ]:
dataframes['cast'].head()

In [ ]:
dataframes['crew'].head()

In [ ]:
def add_popularity_and_budget(df, raw_id, popularity, budget):
    cnt = df.loc[raw_id, 'count']
    pop = df.loc[raw_id, 'popularity']
    bdgt = df.loc[raw_id, 'budget']
    pop = (pop*cnt + popularity)
    bdgt = (bdgt*cnt + budget)
    cnt = cnt+1
    pop = pop/cnt
    bdgt = bdgt/cnt
    df.loc[raw_id, 'count'] = cnt
    df.loc[raw_id, 'popularity'] = pop   
    df.loc[raw_id, 'budget'] = bdgt

In [ ]:
def update_popularity_and_budget(df, column, index_name):
    for i in range(1,len(df)+1):
        if str(df.loc[i, column]) != 'nan':
            temp_list = ast.literal_eval(df.loc[i, column])
            if temp_list:
                temp = pd.DataFrame(temp_list)
                temp = temp.set_index(index_name)
                for j in temp.index:
                    add_popularity_and_budget(dataframes[column], j, df.loc[i,'popularity'], df.loc[i, 'budget'])   

In [ ]:
def update_popularity_and_budget_for_top_5_cast(df, column, index_name):
    for i in range(1,len(df)+1):
        if str(df.loc[i, column]) != 'nan':
            temp_list = ast.literal_eval(df.loc[i, column])
            if temp_list:
                temp = pd.DataFrame(temp_list)
                temp = temp.iloc[:5,:]
                temp = temp.set_index(index_name)
                for j in temp.index:
                    add_popularity_and_budget(dataframes[column], j, df.loc[i,'popularity'], df.loc[i, 'budget'])

In [ ]:
def update_popularity_and_budget_for_directors(df, column, index_name):
    for i in range(1,len(df)+1):
        if str(df.loc[i, column]) != 'nan':
            temp_list = ast.literal_eval(df.loc[i, column])
            if temp_list:
                temp = pd.DataFrame(temp_list)
                temp = temp[temp['job'] == 'Director']
                temp = temp.set_index(index_name)
                for j in temp.index:
                    add_popularity_and_budget(dataframes[column], j, df.loc[i,'popularity'], df.loc[i, 'budget'])

In [ ]:
update_popularity_and_budget(box, 'belongs_to_collection', 'id')
update_popularity_and_budget(box, 'genres', 'id')
update_popularity_and_budget(box, 'production_companies', 'id')
update_popularity_and_budget(box, 'production_countries', 'iso_3166_1')
update_popularity_and_budget(box, 'spoken_languages', 'iso_639_1')
update_popularity_and_budget(box, 'Keywords', 'id')
update_popularity_and_budget_for_top_5_cast(box, 'cast', 'id')
update_popularity_and_budget_for_directors(box, 'crew', 'id')

In [ ]:
# parameters
# column: column name
# entity: 'popularity' or 'budget'
# title: title for plot
# n: top n result, default is 20.
def plot_top_items(column, entity, title, n=20):
    temp = dataframes[column].sort_values(entity, ascending=False).iloc[:n, :]
    plt.rcParams["figure.figsize"] = [16, 14]
    fig, ax = plt.subplots()
    y_pos = np.arange(len(temp))
    ax.barh(y_pos, temp[entity])
    ax.set_yticks(y_pos)
    ax.set_yticklabels(temp['name'])
    ax.invert_yaxis()
    ax.set_xlabel(entity)
    ax.set_title(title)
    plt.show()

In [ ]:
plot_top_items(column='belongs_to_collection', entity='popularity', title='Top 20 popular collections')

In [ ]:
plot_top_items(column='belongs_to_collection', entity='budget', title='Top 20 most expensive collections')

In [ ]:
plot_top_items(column='genres', entity='popularity', title='Genres popularity')

In [ ]:
plot_top_items(column='genres', entity='budget', title='Genres budget')

**Adding new features**

In [ ]:
def add_popularity_budget_features(df, column, index_name):
    df['has_{}'.format(column)] = 0
    df['{}_popularity'.format(column)] = 0
    df['{}_budget'.format(column)] = 0
    for i in range(df.index[0], len(df)+df.index[0]):
#         pop = 0
#         bud = 0
        if str(df.loc[i, column]) != 'nan':
            temp_list = ast.literal_eval(df.loc[i, column])
            if temp_list:
#                 df.loc[i, 'has_{}'.format(column)] = 1
                temp = pd.DataFrame(temp_list)
                temp = temp.set_index(index_name)
                if len(temp) > 0:
                    df.loc[i, '{}_popularity'.format(column)] = dataframes[column].loc[temp.index, 'popularity'].mean()
                    df.loc[i, '{}_budget'.format(column)] = dataframes[column].loc[temp.index, 'budget'].mean()
                    df.loc[i, 'has_{}'.format(column)] = 1
#                 for j in temp.index:
#                     pop = pop + dataframes[column].loc[j, 'popularity']
#                     bud = bud + dataframes[column].loc[j, 'budget']
#                 df.loc[i, '{}_popularity'.format(column)] = pop/len(temp)
#                 df.loc[i, '{}_budget'.format(column)] = bud/len(temp)

In [ ]:
def add_popularity_budget_for_director(df, column, index_name):
    df['has_director'] = 0
    df['director_popularity'] = 0
    df['director_budget'] = 0
    for i in range(df.index[0], len(df)+df.index[0]):
#         pop = 0
#         bud = 0
        if str(df.loc[i, column]) != 'nan':
            temp_list = ast.literal_eval(df.loc[i, column])
            if temp_list:
                temp = pd.DataFrame(temp_list)
                temp = temp[temp['job'] == 'Director']
                temp = temp.set_index(index_name)
#                 has_director = 0
                if len(temp) > 0:
                    df.loc[i, 'director_popularity'] = dataframes[column].loc[temp.index, 'popularity'].mean()
                    df.loc[i, 'director_budget'] = dataframes[column].loc[temp.index, 'budget'].mean()
                    df.loc[i, 'has_director'] = 1
#                 for j in temp.index:
#                     pop = pop + dataframes[column].loc[j, 'popularity']
#                     bud = bud + dataframes[column].loc[j, 'budget']
#                     has_director = 1
#                 df.loc[i, 'director_popularity'] = pop/len(temp)
#                 df.loc[i, 'director_budget'] = bud/len(temp)
#                 df.loc[i, 'has_director'] = has_director

In [ ]:
def add_popularity_budget_for_5_cast(df, column, index_name):
    for n in range(1,6):
        df['has_{}_{}'.format(column, n)] = 0
        df['{}_{}_popularity'.format(column, n)] = 0
        df['{}_{}_budget'.format(column, n)] = 0
    for i in range(df.index[0], len(df)+df.index[0]):
        try:
            if str(df.loc[i, column]) != 'nan':
                temp_list = ast.literal_eval(df.loc[i, column])
                if temp_list:
                    temp = pd.DataFrame(temp_list)
                    if len(temp) > 5:
                        temp = temp.iloc[:5,:]
                        temp = temp.set_index(index_name)
                        cnt = 1
                        for j in temp.index:
                            df.loc[i, 'has_{}_{}'.format(column, cnt)] = 1
                            df.loc[i, '{}_{}_popularity'.format(column, cnt)] = dataframes[column].loc[j, 'popularity']
                            df.loc[i, '{}_{}_budget'.format(column, cnt)] = dataframes[column].loc[j, 'budget']
                            cnt = cnt+1
        except ValueError:
            print('Oops ! Not valid value. Row # ', i)

In [ ]:
from datetime import datetime

def add_date_features(df, column):
    df['release_day'] = 1
    df['release_month'] = 1
    df['release_year'] = 2000
    for i in range(df.index[0], len(df)+df.index[0]):
        if str(df.loc[i, column]) != 'nan':
            release_date = datetime.strptime(str(df.loc[i, column]), '%m/%d/%y')
            df.loc[i, 'release_day'] = release_date.day
            df.loc[i, 'release_month'] = release_date.month
            df.loc[i, 'release_year'] = release_date.year

In [ ]:
from sklearn.preprocessing import OneHotEncoder

def add_catagoriacal_features(df, column):
    df[column].fillna(column+'_unknown', inplace=True)
    cat_encoder = OneHotEncoder(sparse=False, categories='auto', dtype=np.int)
    features = cat_encoder.fit_transform(df[[column]])
    temp_df = pd.DataFrame(features, 
                           index=range(df.index[0], len(df)+df.index[0]), 
                           columns=[column+'_'+str(i) for i in cat_encoder.categories_[0]])
    df = pd.concat([df, temp_df], axis=1)
    return df

In [ ]:
# Preparing data for machine learning Model

box_train_feature = box_train.copy()

# Popularity and budget related features
add_popularity_budget_features(box_train_feature, 'belongs_to_collection', 'id')
add_popularity_budget_features(box_train_feature, 'genres', 'id')
add_popularity_budget_features(box_train_feature, 'production_companies', 'id')
add_popularity_budget_features(box_train_feature, 'production_countries', 'iso_3166_1')
add_popularity_budget_features(box_train_feature, 'spoken_languages', 'iso_639_1')
add_popularity_budget_features(box_train_feature, 'Keywords', 'id')
add_popularity_budget_for_director(box_train_feature, 'crew', 'id')
add_popularity_budget_for_5_cast(box_train_feature, 'cast', 'id')

# Release date related features
add_date_features(box_train_feature, 'release_date')

# Categorical features
# box_train_feature = add_catagoriacal_features(box_train_feature, 'status')
# box_train_feature = add_catagoriacal_features(box_train_feature, 'original_language')
box_train_feature = add_catagoriacal_features(box_train_feature, 'release_day')
box_train_feature = add_catagoriacal_features(box_train_feature, 'release_month')

# Filling not available numerical feature
box_train_feature.runtime.fillna(box_train_feature.runtime.median(), inplace=True)

# Removing un-necessary columns
un_necessary_columns = ['belongs_to_collection', 'genres', 'original_language', 
                        'production_companies', 'production_countries', 'release_date', 
                        'spoken_languages', 'status', 'Keywords', 'crew', 'cast', 
                        'release_day', 'release_month']
box_train_feature.drop(un_necessary_columns, axis=1, inplace=True)

In [ ]:
numerical_ftr = ['budget', 
                 'belongs_to_collection_budget', 'genres_budget', 'production_companies_budget', 
                 'production_countries_budget', 'spoken_languages_budget', 'Keywords_budget', 
                 'cast_1_budget', 'cast_2_budget', 'cast_3_budget', 'cast_4_budget', 'cast_5_budget', 
                 'director_budget',
                 'popularity', 
                 'belongs_to_collection_popularity', 'genres_popularity', 'production_companies_popularity', 
                 'production_countries_popularity', 'spoken_languages_popularity', 'Keywords_popularity', 
                 'cast_1_popularity', 'cast_2_popularity', 'cast_3_popularity', 'cast_4_popularity', 
                 'cast_5_popularity', 'director_popularity', 
                 'runtime', 
                 'revenue']

box_numerical_ftr = box_train_feature[numerical_ftr]
corr_mx_ftr = box_numerical_ftr.corr()
pd.DataFrame(corr_mx_ftr, index=numerical_ftr, columns=numerical_ftr)

In [ ]:
seaborn.heatmap(corr_mx_ftr)
plt.show()

In [ ]:
# Separating features and target

X = box_train_feature.drop('revenue', axis=1)
# X = box_train_feature[['budget', 'popularity']]
y = box_train_feature['revenue']

# from sklearn.decomposition import PCA
# pca = PCA(n_components=0.99)
# X = pca.fit_transform(X)
# print(pca.explained_variance_ratio_)

# Spliting dataset to test and val
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val =  train_test_split(X, y, test_size=0.2)

In [ ]:
# from sklearn.preprocessing import StandardScaler

# std_scaler = StandardScaler()
# X_scaled = std_scaler.fit_transform(X)

In [ ]:
from sklearn.metrics import mean_squared_log_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

def sqrt_mean_squared_log_error(actual, pred):
    return np.sqrt(mean_squared_log_error(actual, pred))

param_grid = [{'n_estimators':[100, 500, 1000], 'max_features':['auto', 'log2'], 
              'min_samples_split':[5, 12, 20], 'max_depth':[5, 20, 50], 'n_jobs':[-1]}]

rf_reg = RandomForestRegressor()

grid = GridSearchCV(rf_reg, param_grid, scoring='neg_mean_squared_log_error', n_jobs=-1)
grid.fit(X, y)

# rf_reg.fit(X_train, y_train)

In [ ]:
rf_reg = grid.best_estimator_

In [ ]:
grid.best_params_

In [ ]:
y_train_pred = rf_reg.predict(X)
y_train_pred[y_train_pred < 0] = 0

error = np.sqrt(mean_squared_log_error(y, y_train_pred))
print(error)
plt.scatter(y, y_train_pred)
plt.show()

In [ ]:
# y_val_pred = rf_reg.predict(X_val)
# y_val_pred[y_val_pred < 0] = 0

# error = np.sqrt(mean_squared_log_error(y_val, y_val_pred))
# print(error)
# plt.scatter(y_val, y_val_pred)
# plt.show()

In [ ]:
box_test_feature = box_test.copy()

# Popularity and budget related features
add_popularity_budget_features(box_test_feature, 'belongs_to_collection', 'id')
add_popularity_budget_features(box_test_feature, 'genres', 'id')
add_popularity_budget_features(box_test_feature, 'production_companies', 'id')
add_popularity_budget_features(box_test_feature, 'production_countries', 'iso_3166_1')
add_popularity_budget_features(box_test_feature, 'spoken_languages', 'iso_639_1')
add_popularity_budget_features(box_test_feature, 'Keywords', 'id')
add_popularity_budget_for_director(box_test_feature, 'crew', 'id')
add_popularity_budget_for_5_cast(box_test_feature, 'cast', 'id')

# Release date related features
add_date_features(box_test_feature, 'release_date')

# Categorical features
# box_test_feature = add_catagoriacal_features(box_test_feature, 'status')
# box_test_feature = add_catagoriacal_features(box_test_feature, 'original_language')
box_test_feature = add_catagoriacal_features(box_test_feature, 'release_day')
box_test_feature = add_catagoriacal_features(box_test_feature, 'release_month')

# Filling not available numerical feature
box_test_feature.runtime.fillna(box_test_feature.runtime.median(), inplace=True)

# Removing un-necessary columns
un_necessary_columns = ['belongs_to_collection', 'genres', 'original_language', 
                        'production_companies', 'production_countries', 'release_date', 
                        'spoken_languages', 'status', 'Keywords', 'crew', 'cast', 
                        'release_day', 'release_month']
box_test_feature.drop(un_necessary_columns+not_useful_columns, axis=1, inplace=True)

In [ ]:
X_test = box_test_feature.copy()

y_test_pred = rf_reg.predict(X_test)
y_test_pred[y_test_pred < 0] = 0
# y_test_pred.shape
# np.array(range(3001, 3001+len(y_test_pred))).reshape()
prediction = pd.DataFrame({'id':np.array(range(3001, 3001+len(y_test_pred))), 'revenue':y_test_pred})
prediction.to_csv('submission.csv', index=False)